In [1]:
from model import Network
from data import load_cifar10_dataloaders, WhiteningTransformation
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from criterion import ReconstructImageFromFCLoss
import numpy as np
from metrics import lpips

# reproducibility
random_seed = 2024

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(random_seed)

import optuna

In [2]:
def objective(trial):
    # load to model
    model_config = './model_config/fc1_cocktail_party_cifar10_instance.json'
    checkpoint_path = './checkpoints/122123_fc1_cocktail_party_cifar10_pretraining_wout_bias_wout_normalization.pth'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = Network(model_config)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)
    
    # get val loader
    batch_size = 8
    data_path = './data'
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    
    _, val_dataloader = load_cifar10_dataloaders(data_path, batch_size, transform)
    selected_val_batch_data, selected_val_batch_label = next(iter(val_dataloader))
    selected_val_batch_data = selected_val_batch_data.to(device)
    selected_val_batch_label = selected_val_batch_label.to(device)
    
    # receiving gradients
    model.zero_grad()
    criterion = nn.CrossEntropyLoss()
    output = model(selected_val_batch_data.reshape(batch_size, -1))
    loss = criterion(output, selected_val_batch_label)
    loss.backward()
    gradient_of_layers = []
    for param in model.parameters():
        gradient_of_layers.append(param.grad.data.clone().to('cpu'))
    print([x.size() for x in gradient_of_layers])
    
    whitening_transform = WhiteningTransformation()
    whitened_gradient = torch.from_numpy(whitening_transform.transform(gradient_of_layers[0].detach().numpy().T)).to(torch.float32).T
    whitened_gradient = whitened_gradient.to(device)
    
    # criterion output testing
    unmixing_matrix = torch.rand((selected_val_batch_data.size(0), gradient_of_layers[0].size(0)), requires_grad=True, device=device)
    t_param = trial.suggest_float('t_param', 0.00001, 10)
    total_variance_loss_param = trial.suggest_float('total_variance_loss_param', 0.00001, 10)
    mutual_independence_loss_param = trial.suggest_float('mutual_independence_loss_param', 0.00001, 10)
    reconstruction_loss = ReconstructImageFromFCLoss(32, 32, 3, t_param, total_variance_loss_param, mutual_independence_loss_param)
    optimizer = torch.optim.Adam([unmixing_matrix])
    
    for iter_idx in range(25000):
        optimizer.zero_grad()
        # out_score, non_gaussianity_score, total_variance_score, mutual_independence_score
        loss, _, _, _ = reconstruction_loss(unmixing_matrix, whitened_gradient)
        loss.backward()
        optimizer.step()
        
        if (iter_idx + 1) % 1000 == 0 or iter_idx == 0:
            print('loss: {}'.format(loss.item()))
            
    unmixing_matrix = unmixing_matrix.detach().to('cpu')
    whitened_gradient = whitened_gradient.detach().to('cpu')
    
    with torch.no_grad():
        lpips_match, lpips_is_positive, mean_lpips = lpips(unmixing_matrix @ whitened_gradient, selected_val_batch_data.detach().to('cpu'), return_matches=True)
        return mean_lpips

In [3]:
num_of_trials = 100
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=num_of_trials)

[I 2023-12-26 02:12:02,094] A new study created in memory with name: no-name-4fca1357-ad86-460d-988f-83fa940a15ca
/home/gulerlab/miniconda3/envs/pytorch-scipy/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([10, 3072])]
loss: 2655.87841796875
loss: 28.589895248413086
loss: 9.468353271484375
loss: 5.071684837341309
loss: 3.195749521255493
loss: 2.32727313041687
loss: 2.16510009765625
loss: 2.1624836921691895
loss: 2.15800142288208
loss: 2.1514627933502197
loss: 2.140873908996582
loss: 2.127697467803955
loss: 2.109002113342285
loss: 2.087414503097534
loss: 2.0647642612457275
loss: 2.042100191116333
loss: 2.021679162979126
loss: 2.0043458938598633
loss: 1.986772060394287
loss: 1.9696383476257324
loss: 1.9565696716308594
loss: 1.9453582763671875
loss: 1.9345190525054932
loss: 1.9268743991851807
loss: 1.9182173013687134
loss: 1.9093987941741943


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/gulerlab/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 114MB/s]  
[I 2023-12-26 02:13:54,332] Trial 0 finished with value: 0.12073554517701268 and parameters: {'t_param': 9.15445638992218, 'total_variance_loss_param': 8.08765090738018, 'mutual_independence_loss_param': 1.7858846165245188}. Best is trial 0 with value: 0.12073554517701268.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([10, 3072])]
loss: 21.06313705444336
loss: 4.779536724090576
loss: 4.486806392669678
loss: 4.486062526702881
loss: 4.4851531982421875
loss: 4.484109401702881
loss: 4.482507228851318
loss: 4.481391906738281
loss: 4.480147838592529
loss: 4.479031562805176
loss: 4.478150367736816
loss: 4.476693153381348
loss: 4.476055145263672
loss: 4.474699974060059
loss: 4.473718166351318
loss: 4.472838878631592
loss: 4.472264766693115
loss: 4.471398830413818
loss: 4.470641613006592
loss: 4.469414710998535
loss: 4.468970775604248
loss: 4.467916965484619
loss: 4.467995643615723
loss: 4.467193126678467
loss: 4.466197490692139
loss: 4.466024398803711


[I 2023-12-26 02:15:40,122] Trial 1 finished with value: 0.12068278901278973 and parameters: {'t_param': 1.9844899314469815, 'total_variance_loss_param': 1.1209112756519888, 'mutual_independence_loss_param': 4.431937115056726}. Best is trial 1 with value: 0.12068278901278973.


...

[I 2023-12-26 05:08:08,840] Trial 97 finished with value: 0.06296581900096498 and parameters: {'t_param': 0.531147228364036, 'total_variance_loss_param': 7.872722824638144, 'mutual_independence_loss_param': 0.4131447477015558}. Best is trial 95 with value: 0.026338792726164684.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([10, 3072])]
loss: 6.188939571380615
loss: 1.0291961431503296
loss: 0.9318904876708984
loss: 0.8452382683753967
loss: 0.7952947020530701
loss: 0.7776557207107544
loss: 0.7684506773948669
loss: 0.7656155228614807
loss: 0.7654195427894592
loss: 0.7646610736846924
loss: 0.7659627199172974
loss: 0.7640676498413086
loss: 0.7639451026916504
loss: 0.7638258337974548
loss: 0.7622358202934265
loss: 0.7615647912025452
loss: 0.7638805508613586
loss: 0.7624021768569946
loss: 0.7630109190940857
loss: 0.7617970705032349
loss: 0.76298588514328
loss: 0.7638660073280334
loss: 0.7626553773880005
loss: 0.7626647353172302
loss: 0.7631449699401855
loss: 0.7628448605537415


[I 2023-12-26 05:09:57,938] Trial 98 finished with value: 0.08294570550788194 and parameters: {'t_param': 2.6686425349843788, 'total_variance_loss_param': 7.55594626288604, 'mutual_independence_loss_param': 0.7435639554655415}. Best is trial 95 with value: 0.026338792726164684.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([10, 3072])]
loss: 4.222973823547363
loss: 1.439503788948059
loss: 1.3120437860488892
loss: 1.2599149942398071
loss: 1.2431704998016357
loss: 1.2371126413345337
loss: 1.234893560409546
loss: 1.2351458072662354
loss: 1.2338402271270752
loss: 1.2331050634384155
loss: 1.232315182685852
loss: 1.2324044704437256
loss: 1.2343218326568604
loss: 1.2326585054397583
loss: 1.234184980392456
loss: 1.2322731018066406
loss: 1.2313868999481201
loss: 1.2311031818389893
loss: 1.2304966449737549
loss: 1.2322636842727661
loss: 1.2325599193572998
loss: 1.2317756414413452
loss: 1.230971097946167
loss: 1.2318637371063232
loss: 1.2329437732696533
loss: 1.2315987348556519


[I 2023-12-26 05:11:47,212] Trial 99 finished with value: 0.13111965940333903 and parameters: {'t_param': 1.4876505635875297, 'total_variance_loss_param': 8.022454974516487, 'mutual_independence_loss_param': 1.2128412458152136}. Best is trial 95 with value: 0.026338792726164684.
